In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import pandas as pd
import numpy as np


In [130]:
# download these libraries if you don't have them
#%pip install PyPortfolioOpt

%pip install PyPortfolioOpt


   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ----- ---------------------------------- 2.6/19.1 MB 12.6 MB/s eta 0:00:02
   ------------ --------------------------- 6.0/19.1 MB 14.8 MB/s eta 0:00:01
   ------------------- -------------------- 9.4/19.1 MB 14.7 MB/s eta 0:00:01
   ------------------------ --------------- 11.5/19.1 MB 13.4 MB/s eta 0:00:01
   ------------------------------- -------- 14.9/19.1 MB 14.0 MB/s eta 0:00:01
   ------------------------------------- -- 18.1/19.1 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 19.1/19.1 MB 13.4 MB/s  0:00:01
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 14.0 MB/s  0:00:00
   ---------------------------------------- 0.0/887.3 kB ? eta -:--:--
   ---------------------------------------- 887.3/887.3 kB 13.3 MB/s  0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   -- ---------------

In [131]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import cvxpy as cp
from pypfopt import BlackLittermanModel



## Group Assignment
### Team Number: 14
### Team Member Names: Aadya, Furqan, Pranay
### Team Strategy Chosen: Market Beat

In [180]:

input_tickers = pd.read_csv("Tickers_Example.csv", names=['Tickers']) #OR any other CSV file with tickers in it
input_tickers

,Tickers
0,AAPL
1,AAPL
2,ABBV
3,ABT
4,ABT
5,ACN
6,HSBC
7,AGN
8,AIG
9,AMZN


In [181]:
## PRELIMINARY DATA PROCESSING 
## AND INPUT PROCESSING

## WILL CSV BE EXACTLY THE WAY IT IS IN EXAMPLE??

## Check if implementation of US and Canadian ticker validation is sufficient/correct
## Should we do it through currency checks, or exchange checks as below?

def validate_ticker(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        if info and 'regularMarketPrice' in info:
            if 'exchange' in info:
                exchange = info['exchange']
                # yfinance uses these exchange codes, so we check against them
                valid_exchanges = ['NMS', 'NGM', 'NYQ','NAD','PCX','BTS','CBQ','ASE', 'TOR', 'TSE', 'Toronto', 'TSXV', 'CNQ', 'CSE','NEO']
                if exchange in valid_exchanges:
                    return True
                else:
                    return False  
            else:
                return False  
        else:
            return False
    except:
        return False

valid_tickers = []
for ticker in input_tickers['Tickers']:
    if validate_ticker(ticker):
        valid_tickers.append(ticker)
        #print(f"✓ Valid: {ticker}")  
    else:
        print(f"✗ Invalid ticker: {ticker}")

valid_tickers_df = pd.DataFrame(valid_tickers, columns=['Ticker'])
valid_tickers_df

# ✗ Invalid ticker: AGN
# ✗ Invalid ticker: MC.PA
# ✗ Invalid ticker: CELG
# ✗ Invalid ticker: 9984.T
# ✗ Invalid ticker: MON
# ✗ Invalid ticker: RELIANCE.NS
# ✗ Invalid ticker: RTN
# ✗ Invalid ticker: INFY.NS
# ✗ Invalid ticker: CPA.AX

✗ Invalid ticker: AGN
✗ Invalid ticker: MC.PA
✗ Invalid ticker: CELG
✗ Invalid ticker: 9984.T
✗ Invalid ticker: MON
✗ Invalid ticker: RELIANCE.NS
✗ Invalid ticker: RTN
✗ Invalid ticker: INFY.NS
✗ Invalid ticker: CPA.AX


,Ticker
0,AAPL
1,AAPL
2,ABBV
3,ABT
4,ABT
5,ACN
6,HSBC
7,AIG
8,AMZN
9,AXP


In [182]:
# check for duplicate tickers

def check_duplicates(df):
    duplicates = df.duplicated()

    for index in duplicates.index:
        if duplicates[index] == True:
            print(f"Removing duplicate: {df.loc[index, 'Ticker']}")
            # remove the row from the dataframe
            df.drop(index, inplace=True)
    return df.reset_index().drop(columns=['index'])

check_duplicates(valid_tickers_df)

Removing duplicate: AAPL
Removing duplicate: ABT
Removing duplicate: BA


,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,HSBC
5,AIG
6,AMZN
7,AXP
8,BA
9,BAC


In [183]:
## REMOVING STOCKS WITH AVG TRADE VOLUME < 5000 between Oct 1, 2024 and Sep 30, 2025 (drop months with < 18 trading days).
##CHECK to ensure, and filter out stocks that do not meet this criteria

def filter_by_avg_volume(df, start_date="2024-10-01", end_date="2025-09-30", min_avg_volume=5000, min_trading_days=18):
    filtered_tickers = []
    for ticker in df['Ticker']:
        stock = yf.Ticker(ticker)
        
        hist = stock.history(start=start_date, end=end_date)
        hist.index = hist.index.tz_localize(None)

        monthly_groups = hist.groupby(hist.index.to_period("M"))
        valid = True
        for month, group in monthly_groups:
            trading_days = len(group)
            if trading_days >= min_trading_days:
                avg_volume = group['Volume'].mean()
                if avg_volume < min_avg_volume:
                    print(f"✗ Ticker failed volume filter: {ticker} (Avg Volume: {avg_volume:.2f} in {month})")
                    valid = False
                    break
        if valid:
            print(f"✓ Ticker passed volume filter: {ticker}")
            filtered_tickers.append(ticker)
    return pd.DataFrame(filtered_tickers, columns=['Ticker'])

final_tickers_df = filter_by_avg_volume(valid_tickers_df)
final_tickers_df


✓ Ticker passed volume filter: AAPL
✓ Ticker passed volume filter: ABBV
✓ Ticker passed volume filter: ABT
✓ Ticker passed volume filter: ACN
✓ Ticker passed volume filter: HSBC
✓ Ticker passed volume filter: AIG
✓ Ticker passed volume filter: AMZN
✓ Ticker passed volume filter: AXP
✓ Ticker passed volume filter: BA
✓ Ticker passed volume filter: BAC
✓ Ticker passed volume filter: BB.TO
✓ Ticker passed volume filter: BIIB
✓ Ticker passed volume filter: BK
✓ Ticker passed volume filter: SAP
✓ Ticker passed volume filter: BLK
✓ Ticker passed volume filter: BMY
✓ Ticker passed volume filter: C
✓ Ticker passed volume filter: CAT
✓ Ticker passed volume filter: CL
✓ Ticker passed volume filter: SAN
✓ Ticker passed volume filter: KO
✓ Ticker passed volume filter: LLY
✓ Ticker passed volume filter: LMT
✓ Ticker passed volume filter: MO
✓ Ticker passed volume filter: MRK
✓ Ticker passed volume filter: PEP
✓ Ticker passed volume filter: PFE
✓ Ticker passed volume filter: PG
✓ Ticker passed volum

,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,HSBC
5,AIG
6,AMZN
7,AXP
8,BA
9,BAC


In [184]:
#Downloading history for valid tickers

tickers_list =  valid_tickers_df['Ticker'].tolist()

stock_data= yf.download(tickers_list, period="5y", interval="1d", group_by='ticker', auto_adjust=True, threads=True)

# Download benchmark data (S&P 500 + TSX average)
sp500_data = yf.download('^GSPC', period='5y', auto_adjust=True)
tsx_data = yf.download('^GSPTSE', period='5y', auto_adjust=True)

# checking the 'Close' prices to ensure they are Series
if isinstance(sp500_data, pd.DataFrame):
    sp500 = sp500_data['Close'].squeeze() 
else:
    sp500 = sp500_data

if isinstance(tsx_data, pd.DataFrame):
    tsx = tsx_data['Close'].squeeze()  
else:
    tsx = tsx_data

# Remove timezones
sp500.index = sp500.index.tz_localize(None)
tsx.index = tsx.index.tz_localize(None)




[*********************100%***********************]  41 of 41 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Scenario 1: High Alpha, Low Beta (< 1.0)

 - Stock outperforms with less volatility than market

 - Best case/ideal scenario 

 - Means it is a Defensive stock that somehow beats the market, low risk high return

Scenario 2: High Alpha, High Beta (> 1.5)

 - Stock outperforms but is very volatile

 - Risky but rewarding

 - Also high reward, but higher risk as well. Still viable option to help choose stocks that will beat market

 - Example: Tech stock in a bull market

Scenario 3: Low/Negative Alpha, High Beta

 - Stock underperforms and is volatile

 - Low return relative to market/less than market, AND high volatility so potential further loss on an already underperforming stock.  

 - Worst case 

 - Want to avoid these types of stocks 


WE WANT TO INVEST IN:

Stocks which fall within the first two scenarios, as they are the most ideal to maximize returns, scenario 1 being the most effective. Stocks which fall in scenario 3, will be avoided. 

In [185]:
## CALCULATING ALPHA AND SORTING TICKERS HIGH TO LOW BY ALPHA


#assuming we have a list of valid tickers
#we need a function that simply returns the alpha of a ticker
#then run a for loop that calls that function for every ticker
#while ranking it from best to worst in a list or dictionary maybe
#going to also assume we have the portfolio returns, risk free rate, expected market return, and the portfolio beta


# calculates beta of a stock 
# beta is cov (Rp, Rm) / var (Rm)
def calc_beta(ticker_returns, market_returns):
    covariance = ticker_returns.cov(market_returns)
    variance = market_returns.var()
    beta = covariance / variance
    return beta

#function to calculate alpha of a ticker given the following parameters
#returns the ticker and its alpha in a list
#Good alpha > 0; Means positive returns relative to the market
#Zero alpha (\(\alpha =0\)): The investment's return was in line with the benchmark index, meaning it did not outperform or underperform after adjusting for risk.
#Negative alpha (\(<0\)): The investment's performance was worse than its benchmark index, considering the risk taken. 
def calc_alpha(portfolio_return, risk_free_rate, expected_market_return, portfolio_beta): 
    tick_alpha_calculation = portfolio_return - (risk_free_rate + (portfolio_beta * (expected_market_return-risk_free_rate)))

    return tick_alpha_calculation

# calculating sharpe ratio 
# sharpe ratio of a stock is = (Rp - Rf) / σp
# Good sharpe ratio > 1: indicates good risk-adjusted performance
# Average sharpe ratio ~ 0.5 - 1: indicates moderate risk-adjusted performance
# Poor sharpe ratio < 0.5: indicates poor risk-adjusted performance
# if ticker_returns are daily, then risk free has to be daily as well
def calc_sharpe(ticker_returns, risk_free_rate):
    std = ticker_returns.std()
    mean_return = ticker_returns.mean()
    sharpe_ratio = (mean_return - risk_free_rate) / std
    return sharpe_ratio

# calculating sortino ratio 
# sortino ratio = (Rp - Rf) / σd when σd is the standard deviation of negative asset returns
# sortino ratio focuses only on downside volatility which helps eliminate 
# the risk of downside movements in asset prices
# Intuitively good return per unit of bad volatility
# Good sortino ratio > 0: good risk-adjusted performance with low downside 
# Zero sortino ratio = 0: no excess return relative to downside risk
# Low sortino ratio < 0: poor risk-adjusted performance with high downside risk

def calc_sortino(ticker_returns, risk_free_rate):
    downside_dev = ticker_returns[ticker_returns < 0].std()
    mean_return = ticker_returns.mean()
    sortino_ratio = (mean_return - risk_free_rate) / downside_dev
    return sortino_ratio


##Calculating momentum, shows how much a stock has returned relative to a chosen date/period a while back
#Determining the change in return a given stock has had over a given period of time can aid in picking
#stocks that are trending upwards


#NOTE TO US, momenutum is only needed if we change the period for other data as well
#for now both are based on 6 months

def calc_momentum(ticker_returns, period):
    momentum = (ticker_returns.iloc[-1]/ticker_returns.iloc[-period]) - 1
    return momentum





In [186]:
sp500_aligned, tsx_aligned = sp500.align(tsx, join='inner')

# Create benchmark
benchmark_prices = (sp500_aligned + tsx_aligned) / 2
benchmark_returns = benchmark_prices.pct_change().dropna()

risk_free_rate_sortino = 0.04 / 252  # Daily risk-free rate assuming 4% annual
risk_free_rate_alpha = 0.04  # Annual risk-free rate for alpha calculation
market_actual_return = (benchmark_prices.iloc[-1] / benchmark_prices.iloc[0]) - 1 

## Will build loop though stock_data[] and as it goes through each ticker we can calculate the relevant metrics for those tickers
##Apply created functions for metrics to the tickers while we iterate through the loop

alpha_results = []

for ticker in tickers_list:
    stock_close = stock_data[ticker]['Close'].dropna()
    stock_close.index = stock_close.index.tz_localize(None)
    stock_returns = stock_close.pct_change().dropna()

    beta = calc_beta(stock_returns, benchmark_returns)

    
    stock_actual_return = (stock_close.iloc[-1] / stock_close.iloc[0]) - 1
    alpha = calc_alpha(stock_actual_return, risk_free_rate_alpha, market_actual_return, beta)

    sortino = calc_sortino(stock_returns, risk_free_rate_sortino)
    momentum = calc_momentum(stock_close, period=63) #3 months
    
    # Append to results list
    alpha_results.append({
        'Ticker': ticker,
        'Alpha': alpha,
        'Beta': beta,
        'Sortino': sortino,
        'Return': stock_actual_return,
        'Momentum': momentum
    })

    print(f"✓ {ticker}: Alpha={alpha:.4f}, Beta={beta:.2f}, Sortino={sortino:.2f}, Momentum={momentum:.2f}")

# Convert to DataFrame
results_df = pd.DataFrame(alpha_results)

# Sort by Alpha (highest first)
results_df = results_df.sort_values('Alpha', ascending=False)

# Display
print("\n=== Top Stocks by Alpha ===")
results_df.reset_index(drop=True, inplace=True)
results_df
    


    

✓ AAPL: Alpha=0.4394, Beta=1.28, Sortino=0.06, Momentum=0.17
✓ ABBV: Alpha=1.4004, Beta=0.40, Sortino=0.06, Momentum=0.11
✓ ABT: Alpha=-0.2459, Beta=0.62, Sortino=0.01, Momentum=-0.06
✓ ACN: Alpha=-0.7993, Beta=1.12, Sortino=0.00, Momentum=-0.06
✓ HSBC: Alpha=1.7720, Beta=1.04, Sortino=0.08, Momentum=0.04
✓ AIG: Alpha=0.2682, Beta=1.15, Sortino=0.05, Momentum=-0.08
✓ AMZN: Alpha=-0.7680, Beta=1.56, Sortino=0.02, Momentum=-0.05
✓ AXP: Alpha=1.0330, Beta=1.49, Sortino=0.07, Momentum=0.09
✓ BA: Alpha=-1.2504, Beta=1.46, Sortino=-0.00, Momentum=-0.21
✓ BAC: Alpha=0.1032, Beta=1.33, Sortino=0.05, Momentum=0.04
✓ BB.TO: Alpha=-1.5426, Beta=1.76, Sortino=0.02, Momentum=0.13
✓ BIIB: Alpha=-0.9679, Beta=0.86, Sortino=-0.02, Momentum=0.23
✓ BK: Alpha=1.2784, Beta=1.21, Sortino=0.08, Momentum=0.04
✓ SAP: Alpha=0.3189, Beta=1.12, Sortino=0.06, Momentum=-0.13
✓ BLK: Alpha=-0.4710, Beta=1.48, Sortino=0.03, Momentum=-0.12
✓ BMY: Alpha=-0.4171, Beta=0.39, Sortino=-0.01, Momentum=-0.01
✓ C: Alpha=0.125

,Ticker,Alpha,Beta,Sortino,Return,Momentum
0,LLY,6.183615,0.614079,0.119881,6.667607,0.502628
1,SAN,2.248017,1.420177,0.088135,3.314834,0.047917
2,HSBC,1.772013,1.036463,0.081773,2.561397,0.043121
3,CAT,1.401320,1.376182,0.083324,2.436327,0.266942
4,ABBV,1.400425,0.398973,0.064058,1.728890,0.113486
5,BK,1.278419,1.205788,0.078186,2.190228,0.037481
6,PM,1.229254,0.458581,0.069572,1.600817,-0.061286
7,AXP,1.032995,1.489888,0.073379,2.150214,0.091422
8,MO,0.830098,0.375002,0.048316,1.141232,-0.118266
9,RY.TO,0.809277,0.769221,0.084611,1.405440,0.099081


In [187]:
##USING METRICS TO PICK TOP 10 STOCKS TO INVEST IN

results_df['Alpha_rank'] = results_df['Alpha'].rank(ascending=False)
results_df['Sortino_rank'] = results_df['Sortino'].rank(ascending=False)
results_df['Momentum_rank'] = results_df['Momentum'].rank(ascending=False)

##Beta close to 1.15 is ideal, so we rank based on distance from 1.15
##ascending True means closest to 1.15 gets rank 1, lowest distance to greatest distance
results_df['Beta_rank'] = (results_df['Beta'] - 1.15).abs().rank(ascending=True)

# Score (simple sum - equal weight)
results_df['Composite_Score'] = (
    results_df['Alpha_rank'] + 
    results_df['Sortino_rank'] + 
    results_df['Momentum_rank'] + 
    results_df['Beta_rank']
)

# Sort and select
# Create new dataframe with just ranks and composite score
scored_df = results_df[['Ticker', 'Alpha_rank', 'Sortino_rank', 'Momentum_rank', 'Beta_rank', 'Composite_Score']].copy()

# Sort by composite score
scored_df = scored_df.sort_values('Composite_Score')

scored_df.reset_index(drop=True, inplace=True)

scored_df


,Ticker,Alpha_rank,Sortino_rank,Momentum_rank,Beta_rank,Composite_Score
0,CAT,4.0,4.0,3.0,12.0,23.0
1,LLY,1.0,1.0,1.0,24.0,27.0
2,HSBC,3.0,5.0,15.0,6.0,29.0
3,SAN,2.0,2.0,14.0,15.0,33.0
4,AAPL,12.0,10.0,5.0,7.0,34.0
5,BK,6.0,6.0,18.0,5.0,35.0
6,RY.TO,10.0,3.0,10.0,20.0,43.0
7,AXP,8.0,7.0,11.0,19.0,45.0
8,TD.TO,11.0,11.0,7.0,21.0,50.0
9,ABBV,5.0,9.0,9.0,31.0,54.0


In [188]:
## TOP 10 STOCKS TO INVEST IN BASED ON METRICS ##
top_10_stocks_beforeCHECK = scored_df.head(10)
top_10_stocks_beforeCHECK


,Ticker,Alpha_rank,Sortino_rank,Momentum_rank,Beta_rank,Composite_Score
0,CAT,4.0,4.0,3.0,12.0,23.0
1,LLY,1.0,1.0,1.0,24.0,27.0
2,HSBC,3.0,5.0,15.0,6.0,29.0
3,SAN,2.0,2.0,14.0,15.0,33.0
4,AAPL,12.0,10.0,5.0,7.0,34.0
5,BK,6.0,6.0,18.0,5.0,35.0
6,RY.TO,10.0,3.0,10.0,20.0,43.0
7,AXP,8.0,7.0,11.0,19.0,45.0
8,TD.TO,11.0,11.0,7.0,21.0,50.0
9,ABBV,5.0,9.0,9.0,31.0,54.0


In [191]:
### OVERALL PORTFOLIO CHECK. MAKING SURE PORTFOLIO MEETS 

### BASIC REQUIREMENTS:
### 1. NO MORE THEN 40% IN A SINGLE SECTOR
### 2. ONE SMALL CAP
### 3. ONE LARGE CAP


## first going to sort the valid tickers into new lists of largecap, small cap, and industry
## which i can then use to check if my portfolio works

#argument should be the list of all valid tickers
def sort_smallcap(tickers):

    smallcap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")

        if market_cap is not None and market_cap < 2000000000:
            smallcap_tickers.append(ticker)

    return smallcap_tickers
     


def sort_largecap(tickers):

    largecap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")
        
        if market_cap is not None and market_cap > 10000000000:
            largecap_tickers.append(ticker)

    return largecap_tickers



#portfolio argument should be a list of just the TEN tickers in the portfolio
################################################# IF WE CHANGE NUMBER OF TICKERS WELL HAVE TO REVISE THIS FUNCTION
#final_tickers argument should be a list of all the valid tickers RANKED from best to worst through all of the analysis above
def check_portfolio(portfolio, final_tickers):
    
    print("\n" + "="*60)
    print("PORTFOLIO VALIDATION CHECK")
    print("="*60)
    print(f"Initial Portfolio: {portfolio}")
    
    smallcap_tickers = sort_smallcap(final_tickers)
    largecap_tickers = sort_largecap(final_tickers)
    
    print(f"\nAvailable Small Cap Stocks (<$2B): {smallcap_tickers}")
    print(f"Available Large Cap Stocks (>$10B): {largecap_tickers}")

    iteration = 0
    
    while True:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        is_smallcap = False
        is_largecap = False
        notfortypercent = True 
        seen_industries = {}
        industry_overforty = None

        for ticker in portfolio:
            if ticker in smallcap_tickers:
                is_smallcap = True
        
            if ticker in largecap_tickers:
                is_largecap = True 

            # FIX: Count industries properly
            industry = yf.Ticker(ticker).info.get("industry")
            seen_industries[industry] = seen_industries.get(industry, 0) + 1

        # Print industry breakdown
        print("\nIndustry Breakdown:")
        for industry, amount in seen_industries.items():
            percentage = (amount / 10) * 100
            print(f"  {industry}: {amount} stocks ({percentage:.0f}%)")
            if amount / 10 > 0.4:
                notfortypercent = False
                industry_overforty = industry

        # Print validation status
        print(f"\n Has Small Cap (<$2B): {'YES' if is_smallcap else 'NO '}")
        print(f" Has Large Cap (>$10B): {'YES' if is_largecap else 'NO '}")
        print(f" No Sector >40%: {'YES' if notfortypercent else f'NO  ({industry_overforty} is over 40%)'}")

        # FIX: Check should be if ALL conditions are TRUE
        if is_smallcap and is_largecap and notfortypercent:
            print("\n" + "="*60)
            print("PORTFOLIO IS VALID!")
            print("="*60)
            print(f"Final Portfolio: {portfolio}")
            return portfolio

        # ACTUALLY FIXING THE PORTFOLIO
        if not is_smallcap:
            print("\n Missing Small Cap stock. Finding replacement...")
            for tick in smallcap_tickers:
                if tick not in portfolio:
                    removed = portfolio.pop(-1)
                    portfolio.append(tick)
                    print(f"   ➜ Removed: {removed}")
                    print(f"   ➜ Added: {tick} (Small Cap)")
                    break
            continue

        if not is_largecap:
            print("\n Missing Large Cap stock. Finding replacement...")
            for tick in largecap_tickers:
                if tick not in portfolio:
                    removed = portfolio.pop(-1)
                    portfolio.append(tick)
                    print(f"   ➜ Removed: {removed}")
                    print(f"   ➜ Added: {tick} (Large Cap)")
                    break
            continue

        if not notfortypercent and industry_overforty is not None:
            print(f"\n Sector '{industry_overforty}' is over 40%. Finding replacement...")
            
            worstposition = -1
            removeworst = None
            swapwith = None

            for tick in portfolio:
                tick_industry = yf.Ticker(tick).info.get("industry")
                if tick_industry == industry_overforty:
                    position = final_tickers.index(tick)
                    if position > worstposition:
                        worstposition = position
                        removeworst = tick

            for tick in final_tickers:
                if tick not in portfolio:
                    tick_industry = yf.Ticker(tick).info.get("industry")
                    if tick_industry != industry_overforty:
                        swapwith = tick
                        break

            if removeworst is not None and swapwith is not None:
                swap_industry = yf.Ticker(swapwith).info.get("industry")
                portfolio.remove(removeworst)
                portfolio.append(swapwith)
                print(f"   ➜ Removed: {removeworst} (Sector: {industry_overforty})")
                print(f"   ➜ Added: {swapwith} (Sector: {swap_industry})")
            continue

        else:
            return portfolio


In [192]:
full_1st = scored_df['Ticker'].tolist()
portfolio_lst = top_10_stocks_beforeCHECK['Ticker'].tolist()

top_10_stocks = check_portfolio(portfolio_lst, full_1st)

##THE LIST HERE should go to the variable top_10_tickers below
#Whatever list is outputted after the check for volume and industry, should move on to the optimization code
#as a list and set equal to top_10_tickers



PORTFOLIO VALIDATION CHECK
Initial Portfolio: ['CAT', 'LLY', 'HSBC', 'SAN', 'AAPL', 'BK', 'RY.TO', 'AXP', 'TD.TO', 'ABBV']

Available Small Cap Stocks (<$2B): ['FLGT']
Available Large Cap Stocks (>$10B): ['CAT', 'LLY', 'HSBC', 'SAN', 'AAPL', 'BK', 'RY.TO', 'AXP', 'TD.TO', 'ABBV', 'BAC', 'AIG', 'C', 'SAP', 'PM', 'MRK', 'LMT', 'KO', 'UNP', 'USB', 'BIIB', 'MO', 'UPS', 'ACN', 'BLK', 'AMZN', 'ABT', 'UNH', 'QCOM', 'PEP', 'TXN', 'SHOP.TO', 'PG', 'BMY', 'PFE', 'CL', 'BA', 'T.TO', 'PYPL']

--- Iteration 1 ---

Industry Breakdown:
  Farm & Heavy Construction Machinery: 1 stocks (10%)
  Drug Manufacturers - General: 2 stocks (20%)
  Banks - Diversified: 5 stocks (50%)
  Consumer Electronics: 1 stocks (10%)
  Credit Services: 1 stocks (10%)

 Has Small Cap (<$2B): NO 
 Has Large Cap (>$10B): YES
 No Sector >40%: NO  (Banks - Diversified is over 40%)

 Missing Small Cap stock. Finding replacement...
   ➜ Removed: ABBV
   ➜ Added: FLGT (Small Cap)

--- Iteration 2 ---

Industry Breakdown:
  Farm & 

In [167]:
# Get the top 10 tickers
top_10_tickers = top_10_stocks['Ticker'].tolist()

# Get the actual metric values for top 10 stocks from results_df
top_10_metrics = results_df[results_df['Ticker'].isin(top_10_tickers)].copy()

# Build prices DataFrame for top 10 stocks
prices_df = pd.DataFrame()
for ticker in top_10_tickers:
    stock_close = stock_data[ticker]['Close'].dropna()
    stock_close.index = stock_close.index.tz_localize(None)
    prices_df[ticker] = stock_close

# Align all columns to same dates
prices_df = prices_df.dropna()
prices_df


,CAT,LLY,HSBC,SAN,AAPL,BK,RY.TO,AXP,TD.TO,ABBV
Date,,,,,,,,,,
2020-11-23,158.928375,136.064606,18.970083,2.331492,110.827538,33.370651,87.514854,109.135452,55.992432,84.081818
2020-11-24,160.856110,137.507019,19.747847,2.484099,112.112503,35.223133,89.263489,113.177826,57.099445,85.546547
2020-11-25,158.792023,137.364731,19.814514,2.467142,112.949661,34.946125,89.263489,113.356461,57.011208,85.744034
2020-11-27,159.201187,139.917465,19.918217,2.467142,113.494804,34.582558,89.636429,113.365868,56.874825,86.311813
2020-11-30,157.846313,138.218796,19.177486,2.390839,115.889503,33.864059,87.896072,111.485664,55.575298,86.056725
...,...,...,...,...,...,...,...,...,...,...
2025-11-14,554.030029,1025.280029,72.559998,10.750000,272.410004,110.480003,205.460007,357.179993,114.050003,232.360001
2025-11-17,552.049988,1021.700012,71.220001,10.400000,267.459991,106.769997,206.929993,341.250000,114.459999,234.000000
2025-11-18,546.880005,1030.050049,69.589996,10.170000,267.440002,107.500000,208.380005,340.660004,114.930000,233.869995


In [ ]:
print(f"\n{'='*60}")
print("BLACK-LITTERMAN PORTFOLIO OPTIMIZATION")
print(f"{'='*60}")

# Calculate market baseline
years = 5
market_total_return = (benchmark_prices.iloc[-1] / benchmark_prices.iloc[0]) - 1
market_baseline = (1 + market_total_return) ** (1/years) - 1

print(f"Market Baseline Annual Return: {market_baseline:.2%}") 

# Create views and confidences
viewdict = {}
confidences = []

for _, row in top_10_metrics.iterrows():
    ticker = row['Ticker']
    alpha_annual = row['Alpha']
    momentum_3mo = row['Momentum']
    momentum_annual = (1 + momentum_3mo) ** 4 - 1
    
    expected_return = market_baseline + alpha_annual + (0.5 * momentum_annual)
    expected_return = np.clip(expected_return, 0.05, 0.30)
    viewdict[ticker] = expected_return
    
    alpha_norm = row['Alpha'] / top_10_metrics['Alpha'].max()
    sortino_norm = row['Sortino'] / top_10_metrics['Sortino'].max()
    combined = (alpha_norm + sortino_norm) / 2
    confidence = 0.5 + combined * 0.5
    confidences.append(confidence)

# Black-Litterman with prior
S = risk_models.sample_cov(prices_df)

bl = BlackLittermanModel(
    S, 
    absolute_views=viewdict, 
    view_confidences=confidences,
    pi='equal'  # FIX 1: Add prior
)

ret_bl = bl.bl_returns()
S_bl = bl.bl_cov()

# Optimize
risk_free = 0.04
ef = EfficientFrontier(ret_bl, S_bl, weight_bounds=(0.05, 0.15))
weights = ef.max_sharpe(risk_free_rate=risk_free)
cleaned_weights = ef.clean_weights()

# Get performance with same risk-free rate
performance = ef.portfolio_performance(
    verbose=False, 
    risk_free_rate=risk_free  # FIX 2: Use same rate
)

expected_return, expected_volatility, sharpe_ratio = performance

# Create DataFrame
weights_df = pd.DataFrame({
    'Ticker': list(cleaned_weights.keys()),
    'Weight': list(cleaned_weights.values())
}).sort_values('Weight', ascending=False)

weights_df['Weight'] = weights_df['Weight'] / weights_df['Weight'].sum()

print(f"\n{'='*60}")
print("OPTIMIZED PORTFOLIO WEIGHTS")
print(f"{'='*60}")
print(weights_df.to_string(index=False))

print(f"\nExpected Annual Return:    {expected_return:.2%}")
print(f"Expected Annual Volatility: {expected_volatility:.2%}")
print(f"Expected Sharpe Ratio:      {sharpe_ratio:.2f}")


BLACK-LITTERMAN PORTFOLIO OPTIMIZATION
Annual Return: 12.01%

OPTIMIZED PORTFOLIO WEIGHTS
Ticker   Weight
 RY.TO 0.149999
 TD.TO 0.149999
  ABBV 0.149999
   LLY 0.113919
  AAPL 0.102019
  HSBC 0.099379
    BK 0.084689
   CAT 0.050000
   SAN 0.050000
   AXP 0.050000

Expected Annual Return:    25.40%
Expected Annual Volatility: 14.82%
Expected Sharpe Ratio:      1.44


In [ ]:
''''

### OVERALL PORTFOLIO CHECK. MAKING SURE PORTFOLIO MEETS 

### BASIC REQUIREMENTS:
### 1. NO MORE THEN 40% IN A SINGLE SECTOR
### 2. ONE SMALL CAP
### 3. ONE LARGE CAP


## first going to sort the valid tickers into new lists of largecap, small cap, and industry
## which i can then use to check if my portfolio works

#argument should be the list of all valid tickers
def sort_smallcap(tickers):

    smallcap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")

        if market_cap is not None and market_cap < 2000000000:
            smallcap_tickers.append(ticker)

    return smallcap_tickers
     


def sort_largecap(tickers):

    largecap_tickers = []

    for ticker in tickers:

        tick_info = yf.Ticker(ticker).fast_info
        market_cap = tick_info.get("marketCap")
        
        if market_cap is not None and market_cap > 10000000000:
            largecap_tickers.append(ticker)

    return largecap_tickers



#portfolio argument should be a list of just the TEN tickers in the portfolio
################################################# IF WE CHANGE NUMBER OF TICKERS WELL HAVE TO REVISE THIS FUNCTION
#final_tickers argument should be a list of all the valid tickers RANKED from best to worst through all of the analysis above
def check_portfolio(portfolio, final_tickers):
    

    smallcap_tickers = sort_smallcap(final_tickers)
    largecap_tickers = sort_largecap(final_tickers)

    while True:

        is_smallcap = False
        is_largecap = False
        notfortypercent = True 
        seen_industries = {}
        industry_overforty = None



        for ticker in portfolio:

            if ticker in smallcap_tickers:
                is_smallcap = True
        
            if ticker in largecap_tickers:
                is_largecap = True 

            #keeping track of how many times weve seen a certain industry
            industry = yf.Ticker(ticker).info.get("industry")
            seen_industries[industry] += 1

        for industry, amount in seen_industries.items():
            if amount / 10 > 0.4:
                notfortypercent = False
                industry_overforty = industry

        if is_smallcap and is_largecap and industry_overforty:
            print("Portfolio is Valid.")
            return portfolio
        


        #ACTUALLY FIXING THE PORTFOLIO

        #the logic for small cap and large cap is this:
        #if our booleans are true then there is NO small/large cap in our portfolio so
        #we remove the last stock in our actual portfolio, because 1-9 are better then the tenth stock based of our algorithm
        #then we take the first ticker in our sorted list of small/large cap tickers that is NOT in our portfolio, 
        # (technically there shouldnt be a small/large cap in there but this is a good extra safet check)
        #and we put it in to the portfolio back at the 10th position

        if not is_smallcap:
            for tick in smallcap_tickers:
                if tick not in portfolio:
                    portfolio.pop(-1)
                    portfolio.append(tick)
                    break

            continue

        if not is_largecap:
            for tick in largecap_tickers:
                if tick not in portfolio:
                    portfolio.pop(-1)
                    portfolio.append(tick)
                    break

            continue


        #the logic for the industry check is this:
        #if there is an industry worth over forty percent AND we have the industry name, which we should then
        #we take out the last occurene of that industry in our portfolio, again because its ranked high to low, the latter stocks are non-optimal picks
        #so we remove the last occurece, and then we look in the tickers for the first one that is NOT in the portfolio and NOT in the same industry
        #and then we replace it by appending it to the end of the portfolio
        
    
        if not notfortypercent and industry_overforty is not None:

            worstposition = -1 #last index
            removeworst = None
            swapwith = None

            for tick in portfolio:

                tick_industry = yf.Ticker(tick).info.get("industry")

                if tick_industry == industry_overforty:
                    position = final_tickers.index(tick)

                    if position > worstposition:
                        worstposition = position
                        removeworst = tick

        
            for tick in final_tickers:
                if tick not in portfolio:
                    tick_industry = yf.Ticker(tick).info.get("industry")

                    if tick_industry != industry_overforty:
                        swapwith = tick
                        break


            if removeworst != None and swapwith != None:
                portfolio.remove(removeworst)
                portfolio.append(swapwith)

            continue


        else:
            return portfolio
        
'''




## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.